#### Importing Required Modules

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input,Dropout

#### Reading and Normalizing the data using Pandas


In [8]:
data = pd.read_csv("stock_data.csv")
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by='Date').reset_index(drop=True)
missing_close_data = data[data['Close'].isna()][['Date']].copy()
data[['Open', 'High', 'Low', 'Volume']] = data[['Open', 'High', 'Low', 'Volume']].ffill()
train = data.dropna(subset=['Close']).copy()
feature_cols = ['Open', 'High', 'Low', 'Volume']
scaler = MinMaxScaler()
train[feature_cols] = scaler.fit_transform(train[feature_cols])
true_close_scaler = MinMaxScaler()
train[['Close']] = true_close_scaler.fit_transform(train[['Close']])

#### Making Time series Sequences

In [9]:
def sequences(data, seq_length=7):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[feature_cols].iloc[i:i+seq_length].values)
        y.append(data['Close'].iloc[i+seq_length])
    return np.array(x), np.array(y)
x_train, y_train = sequences(train)

#### Model Architecture and Training

In [10]:
model = Sequential([
    Input(shape=(7,4)),
    LSTM(64, activation='tanh'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.summary()
model.fit(x_train, y_train, epochs=100, batch_size=16)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 64)             │        17,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,777 (77.25 KB)

 Trainable params: 19,777 (77.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1989  
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0380 
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0252 
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0330 
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0255 
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0263 
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0246 
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0205 
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0264 
Epoch 10/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0210 
Epoch 11/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0159 
Epoch 12/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0225 
Epoch 13/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0203 
Epoch 14/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0163 
Epoch 15/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - lo

#### Prediction of the Close of missing dates


In [11]:
predicted = []
valid_dates = []

last_known_close_actual = true_close_scaler.inverse_transform(
    [[train['Close'].iloc[-1]]]
)[0][0]


date_indices = {date: data[data['Date'] == date].index[0] 
               for date in missing_close_data['Date']}

for date, index in date_indices.items():

    if index < 10:
        predicted.append(round(last_known_close_actual, 2))
        valid_dates.append(date)
        continue

    prev_window = data.iloc[index-10:index]

    if prev_window[feature_cols].isnull().values.any():
        predicted.append(round(last_known_close_actual, 2))
        valid_dates.append(date)
        continue

    try:

        input_seq = scaler.transform(prev_window[feature_cols])
        input_seq = np.expand_dims(input_seq, axis=0)
        

        pred = model.predict(input_seq, verbose=0)
        pred_actual = true_close_scaler.inverse_transform(pred)[0][0]
        
        predicted.append(round(pred_actual, 2))
        valid_dates.append(date)
    except Exception:
        predicted.append(round(last_known_close_actual, 2))
        valid_dates.append(date)

#### Saving the Required CSV

In [12]:
result_data = missing_close_data.loc[
    missing_close_data['Date'].isin(valid_dates)].copy()
result_data['Predicted_Close'] = predicted
result_data.to_csv("close.csv", index=False)